# Cargar y guardar pesos de modelos en Pytorch

Durante toda la sección 5 se ha estado evaluando   numéricamente  el  progreso  del  entrenamiento  y  preentrenar  un  LLM  desde  cero.   Si  bien  tanto  el  LLM  como  el  conjunto  de  datos  eran  relativamente  pequeños,  este  ejercicio  demostró  que  preentrenar  LLM  es  computacionalmente  costoso.  Por  lo  tanto,  es  importante  poder  guardar  el  LLM  para  no  tener  que  volver  a  ejecutar  el  entrenamiento  cada  vez.

![Texto alternativo](./imgs/5.14.png)

En esta sección se explicará s  cómo  guardar  y  cargar  un  modelo  preentrenado.

In [5]:
import sys
import os

# Obtiene la ruta de la carpeta principal del proyecto (subiendo un nivel desde seccion05)
ruta_proyecto_principal = os.path.abspath(os.path.join(os.getcwd(), '..'))

# Añade esta ruta a la lista de lugares donde Python busca módulos
if ruta_proyecto_principal not in sys.path:
    sys.path.append(ruta_proyecto_principal)

In [6]:
import torch
import torch.nn as nn
import tiktoken
from seccion04_ImplementacionGPTGeneracionTexto.gptModel import GPTModel
from seccion04_ImplementacionGPTGeneracionTexto.gptConfig124M import GPT_CONFIG_124M
from seccion02_TrabajarDatosTexto.dataloader_v1 import create_dataloader_v1
from trainModelSimple import train_model_simple


GPT_CONFIG_124M["context_length"] = 256

file_path = "../txt/The_Verdict.txt"
with open(file_path, "r", encoding="utf-8") as f:
    text_data = f.read()

tokenizer = tiktoken.get_encoding("gpt2")

train_ratio = 0.90
split_idx = int(train_ratio * len(text_data))
train_data = text_data[: split_idx]
val_data = text_data[split_idx :]

torch.manual_seed(123)
train_loader = create_dataloader_v1(
    train_data,
    batch_size=2,
    max_length=GPT_CONFIG_124M["context_length"],
    stride=GPT_CONFIG_124M["context_length"],
    drop_last=True,
    shuffle=True,
    num_workers=0
)
val_loader = create_dataloader_v1(
    val_data,
    batch_size=2,
    max_length=GPT_CONFIG_124M["context_length"],
    stride=GPT_CONFIG_124M["context_length"],
    drop_last=False,
    shuffle=False,
    num_workers=0
)

device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
tokenizer = tiktoken.get_encoding("gpt2")

torch.manual_seed(123)
model = GPTModel(GPT_CONFIG_124M)
model.to(device)
optimizer = torch.optim.AdamW(model.parameters(), lr=0.0004, weight_decay=0.1) #el método parameters() devuelve todos los pesos entrenables del modelo
num_epochs = 10
train_losses, val_losses, tokens_seen = train_model_simple(
    model, train_loader, val_loader, optimizer, device,
    num_epochs=num_epochs, eval_freq=5, eval_iter=1,
    start_context="Every effort moves you", tokenizer=tokenizer
)

Ep 1 (Step 000000): Train loss 10.063, Val loss 9.945
Ep 1 (Step 000005): Train loss 8.156, Val loss 8.334
Every effort moves you,,,,,,,,,,,,.                                     
Ep 2 (Step 000010): Train loss 6.556, Val loss 7.037
Ep 2 (Step 000015): Train loss 5.927, Val loss 6.580
Every effort moves you, and,, and,,,,,,, and,.                                   
Ep 3 (Step 000020): Train loss 5.824, Val loss 6.464
Ep 3 (Step 000025): Train loss 5.381, Val loss 6.365
Every effort moves you, and to the to the of the to the, and I had. Gis, and, and, and, and, and, and I had, and, and, and, and, and, and, and, and, and,
Ep 4 (Step 000030): Train loss 4.598, Val loss 6.250
Ep 4 (Step 000035): Train loss 5.028, Val loss 6.267
Every effort moves you of the picture.      "I                "I"I the picture"I had the picture"I the picture and I had been the picture of
Ep 5 (Step 000040): Train loss 4.076, Val loss 6.136
Every effort moves you know the                          "Oh, and the fa

Después  de  entrenar  e  inspeccionar  el  modelo,  a  menudo  es  útil  guardar  el  modelo  para  que  podamos  usarlo  o  continuar  entrenándolo  más  tarde.

Afortunadamente,  guardar  un  modelo  de  PyTorch  es  relativamente  sencillo.  La  forma  recomendada  es  guardar  el  llamado  state_dict  del  modelo,  un  diccionario  que  asigna  cada  capa  a  sus  parámetros,  usando  la  función  torch.save  de  la  siguiente  manera:

In [7]:
torch.save(model.state_dict(), "model.pth")

En  el  código  anterior,  "model.pth"  es  el  nombre  del  archivo  donde  se  guarda  el  state_dict .  La  extensión .pth  es  una  convención  para  los  archivos  de  PyTorch,  aunque  técnicamente  se puede  usar  cualquier extensión.

Luego después de guradar lo pesos del modelo a través de state_dict, se puede cargar los pesos del modelo en una nueva instancia de modelo GPTModel de la siguiente manera.

In [8]:
model2 = GPTModel(GPT_CONFIG_124M)
model2.load_state_dict(torch.load("model.pth"))
model.eval()

GPTModel(
  (tok_emb): Embedding(50257, 768)
  (pos_emb): Embedding(256, 768)
  (drop_emb): Dropout(p=0.1, inplace=False)
  (trf_blocks): Sequential(
    (0): TransformerBlock(
      (att): MultiHeadAttention(
        (W_query): Linear(in_features=768, out_features=768, bias=False)
        (W_key): Linear(in_features=768, out_features=768, bias=False)
        (W_value): Linear(in_features=768, out_features=768, bias=False)
        (out_proj): Linear(in_features=768, out_features=768, bias=True)
        (dropout): Dropout(p=0.1, inplace=False)
      )
      (ff): FeedForward(
        (layers): Sequential(
          (0): Linear(in_features=768, out_features=3072, bias=True)
          (1): GELU()
          (2): Linear(in_features=3072, out_features=768, bias=True)
        )
      )
      (norm1): LayerNorm()
      (norm2): LayerNorm()
      (drop_shortcut): Dropout(p=0.1, inplace=False)
    )
    (1): TransformerBlock(
      (att): MultiHeadAttention(
        (W_query): Linear(in_features

Como se describio en la sección 4,  la  función  de  abandono  evita  que  el  modelo  se  sobreajuste  a  los  datos  de  entrenamiento  eliminando  aleatoriamente  neuronas  de  una  capa  durante  el  entrenamiento.  Sin  embargo,  durante  la  inferencia,  no  se quiere eliminar  aleatoriamente  la  información  que  la  red  ha  aprendido.  El  uso  de  model.eval()  cambia  el  modelo  al  modo  de  evaluación  para  la  inferencia,  deshabilitando  las  capas  de  abandono  del  modelo.

Si se plantea continuar  entrenando  previamente  un  modelo  más  adelante,  por  ejemplo,  utilizando  la  función  train_model_simple  que  definimos  anteriormente  en  este  capítulo,  también  se  recomienda  guardar  el  estado  del  optimizador.

Los  optimizadores  adaptativos  como  AdamW  almacenan  parámetros  adicionales  para  cada  peso  del  modelo.AdamW  utiliza  datos  históricos  para  ajustar  dinámicamente  las  tasas  de  aprendizaje  de  cada  parámetro  del  modelo.
Sin  él,  el  optimizador  se  reinicia  y  el  modelo  podría  aprender  de  forma  subóptima  o  incluso  no  converger  correctamente,  lo  que  significa  que  perderá  la  capacidad  de  generar  texto  coherente.

In [9]:
torch.save({
    "model_state_dict": model.state_dict(),
    "optimizer_state_dict": optimizer.state_dict(),
    },
    "model_and_optimizer.pth"
)

Para restaurar los  estados  del  modelo  y  del  optimizador  de  la  siguiente  manera:  primero  cargando  los  datos  guardados  a  través  de  torch.load  y  luego  usando  el  método  load_state_dict :

In [12]:
checkpoint = torch.load("model_and_optimizer.pth")
model3 = GPTModel(GPT_CONFIG_124M)
model3.load_state_dict(checkpoint["model_state_dict"])
optimizer3 = torch.optim.AdamW(model.parameters(), lr=5e-4, weight_decay=0.1)
optimizer3.load_state_dict(checkpoint["optimizer_state_dict"])
model3.train();

In [13]:
model3.to(device)
num_epochs = 1
train_losses, val_losses, tokens_seen = train_model_simple(
    model3, train_loader, val_loader, optimizer3, device,
    num_epochs=num_epochs, eval_freq=5, eval_iter=1,
    start_context="Every effort moves you?  Yes--quite insensible to the irony. She wanted him vindicated--and by me!  He laughed again, and threw back his head to look up at the sketch of the donkey. There were days when I", tokenizer=tokenizer
)



Ep 1 (Step 000000): Train loss 0.449, Val loss 6.429
Ep 1 (Step 000005): Train loss 0.621, Val loss 6.429
Every effort moves you?  Yes--quite insensible to the irony. She wanted him vindicated--and by me!  He laughed again, and threw back his head to look up at the sketch of the donkey. There were days when I had been dead."    "--and I was posing to myself like one of my own sitters.  "Oh, he had been dead." "Oh, he was a sketch of a donkey--an old tired donkey


[Carga de pesos preentrenados desde OpenAI](./5_carga_pesos_preentrenados_openAI.ipynb)